***STEPS:***

1. Collect "Negative" or "Background images"
      - Any image without the object (we need), thousands of them.

2. Collect or create "Positive" images
      - Thousands of images of the object.

3. Create a positive vector file by stitching all the positives.
      - Done with OpenCV command.

4. Train Cascade
      - Done with OpenCV command.



---

**Negative Images:** bg.txt file contains path to each image, by line.

ex: neg/1.jpg

**Positive Images:** pos.txt contains path to each image, along with how many objects, and where they are located

ex: pos/1.jpg 1 0 0 50 50  => image, no of objs, rectange coordinates.



---

negative images > positive images.

Try small images. 100 x 100 -> negatives, 50 x 50 -> positives.

For trainig we use even smaller images (20 x 20).

Have ~ double positive images compared to negative images.

In [0]:
!mkdir opencv_workspace
import os
os.chdir('opencv_workspace/')

In [3]:
!git clone https://github.com/Itseez/opencv.git

Cloning into 'opencv'...
remote: Enumerating objects: 2, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 251159 (delta 0), reused 0 (delta 0), pack-reused 251157
Receiving objects: 100% (251159/251159), 457.09 MiB | 25.78 MiB/s, done.
Resolving deltas: 100% (175181/175181), done.
Checking out files: 100% (5801/5801), done.


In [0]:
# download images by link
import urllib.request
import cv2
import numpy as np

def store_raw_images():
  neg_images_link = 'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n00523513'
  neg_image_urls = urllib.request.urlopen(neg_images_link).read().decode()
  pic_num = 1

  if not os.path.exists('neg'):
     os.makedirs('neg')
     
  for i in neg_image_urls.split('\n'):
      try:
        urllib.request.urlretrieve(i,"neg/"+str(pic_num)+'.jpg')
        img = cv2.imread("neg/"+str(pic_num)+'.jpg', cv2.IMREAD_GRAYSCALE)
        resized_image = cv2.resize(img, (100, 100))
        cv2.imwrite("neg/"+str(pic_num)+".jpg",resized_image)
        pic_num += 1
      except Exception as e:
        (str(e))

In [0]:
store_raw_images()

In [0]:
# copy the unecessary image pattern in the uglies folder and run the script
def find_uglies():
    match = False
    for file_type in ['neg']:
        for img in os.listdir(file_type):
            for ugly in os.listdir('uglies'):
                try:
                    current_image_path = str(file_type)+'/'+str(img)
                    ugly = cv2.imread('uglies/'+str(ugly))
                    question = cv2.imread(current_image_path)
                    if ugly.shape == question.shape and not(np.bitwise_xor(ugly,question).any()):
                        print('That is one ugly pic! Deleting!')
                        print(current_image_path)
                        os.remove(current_image_path)
                except Exception as e:
                    print(str(e))

In [0]:
# create the descriptor files
def create_pos_n_neg():
    for file_type in ['neg']:
        
        for img in os.listdir(file_type):

            if file_type == 'pos':
                line = file_type+'/'+img+' 1 0 0 50 50\n'
                with open('info.dat','a') as f:
                    f.write(line)
            elif file_type == 'neg':
                line = file_type+'/'+img+'\n'
                with open('bg.txt','a') as f:
                    f.write(line)

In [0]:
# pos images in info and data for cascade information
!mkdir data info

In [0]:
# We're ready to create some positive samples now, based on the watch5050.jpg image. To do this, run the following via the terminal, while in the workspace:
opencv_createsamples -img watch5050.jpg -bg bg.txt -info info/info.lst -pngoutput info -maxxangle 0.5 -maxyangle 0.5 -maxzangle 0.5 -num 1950

In [0]:
#create the vector file, which is basically where we stitch all of our positive images together

opencv_createsamples -info info/info.lst -num 1950 -w 20 -h 20 -vec positives.vec

In [0]:
nohup opencv_traincascade -data data -vec positives.vec -bg bg.txt -numPos 1800 -numNeg 900 -numStages 10 -w 20 -h 20 &

In [0]:
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

#this is the cascade we just made. Call what you want
watch_cascade = cv2.CascadeClassifier('watchcascade10stage.xml')

cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    watches = watch_cascade.detectMultiScale(gray, 50, 50)
    
    # add this
    for (x,y,w,h) in watches:
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img,'Watch',(x-w,y-h), font, 0.5, (11,255,255), 2, cv2.LINE_AA)
        #cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()